# Baseline Model
This is a first shot at generation blogposts. This model is meant to be set up easy and quick and produce some result, not necessaryly usable text. I will try it GPT2 first, maybe also something else. 


## GPT2 only, no fine-tuning
htis one is really easy to set up, we basically only need the pipeline. The task at hand is text generation, and our model will be gpt2. 

In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 34.9 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from transformers import pipeline
blogPoster = pipeline(task='text-generation', model='distilgpt2')

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

We can now call this with no arguments or an empty string as a starting point and it will give some text. However, it won't per default write about NFTs but about a wide range of topics from the texts, it has been trained on. 

To get an idea, we can generate a few of those. 

In [12]:
blogPoster("", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' and\n\n\n“ I wish I were there, but you know, it\'s too dangerous, you know, that\'s it I\'m supposed to be," said Mr. J. F. J. and Mr. K. J.'},
 {'generated_text': ', and the second half of the game, the players did a good job of finding two turnovers.'},
 {'generated_text': " I was expecting something like this from a future post.\nThe next installment doesn't have any specific features but will include a few things that will allow you to make it much easier to connect with friends and family.\nI'm also going to"},
 {'generated_text': 'The United States and its allies should be on the same page as China, it is expected to cut emissions by nearly half a billion metric tons by 2050, from an environmental level by more than a billion, said James M. Schmitt, lead'},
 {'generated_text': 'Seth Miller, co-author of a book on the book, explains how it should be seen as a more comprehensive look at the role the U.S. government is playing in the war on terror.\n\n\n\n\n\n

We can push it a bit in the right direction by giving it something to start with e.g. "The next NFT-based..."

In [4]:
blogPoster("The next NFT-based", num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The next NFT-based event will be NFT-2X11R by ENS. ENS has released its first NFT-8LX15K and NFT-8LX15R NFT-5LX15'},
 {'generated_text': 'The next NFT-based NFT business will be in London in March 2018. This business has been going by the name of "The Pico-McDonald" and is primarily owned by its UK employees, many of which are new to the'},
 {'generated_text': "The next NFT-based news network will be NTR.\n\nFinnish Premier Li Keqiang is also scheduled to visit the state later this month, so expect a meeting at the party's office in Oslo later this week.\n"},
 {'generated_text': 'The next NFT-based organization would be the World Wildlife Fund, but its name has been changed, and is likely to remain a separate entity. The same organization is called the U.S. Fish & Wildlife Service, although it is under federal'},
 {'generated_text': 'The next NFT-based service provider, based at the same headquarters of Koryakuland Police Department, is the NFT-Pepsi Corp. in Banga

Ok, so that was quite bad, as was to be expected, probably. I think we will have to do some fine-tuning.

## Fine-tune GPT2 with the NFT articles data set 
The idea is to take the gpt2-model and train it a few epochs on the NFT articles data set. This will make it more probable to genereate relevant output without the need for a full training from scratch. So we use transfer learning to get there faster. This approach is called domain adaption. So we want to adapt the gpt2-model to the domain of creating NFT-related articles. 

There are two tutorials to follow here: 
https://huggingface.co/course/chapter7/6?fw=pt
This one is on Training a causal language model from scratch, and the other one https://huggingface.co/course/chapter3/3?fw=tf
is on fine-tuning a pretrained model. Those two can be combined to give the desired result, hopefully. 

## Load the data set
First, we need to load the data set into colab. I found a way to do this, however, this requires manual input of the data file. I am shure there are more elegant ways to do this. 

The file to upload is "articles.json". 

In [7]:
from google.colab import files
uploaded = files.upload()

Saving articles.json to articles.json


We can now read the file and get access to the articles. This gives us the "trainingArticles" variable, which is a list of articles that we want to use for domain adaption. 

In [8]:
import json

myfile = open('articles.json', 'rb')
trainingArticles = json.load(myfile)
myfile.close()

In [ ]:
print(trainingArticles[1])

## Load the model and tokenizer
Now we import the nessessary modules. We will use the AutoTokenizer and AutoModelForCausalLM to simply use the tokenizer and model from the pretrained model. We will start with the "distilgpt2"-model. 

In [49]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForCausalLM

checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForCausalLM.from_pretrained(checkpoint)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.summary()

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 81912576  
 r)                                                              
                                                                 
Total params: 81,912,576
Trainable params: 81,912,576
Non-trainable params: 0
_________________________________________________________________


## Prepare the data for training
Here we prepare the data for input into the model for training. 

My first approach was to just use the tokenizer to tokenize the articles and then call model.fit with the tokenized data for training. however, this naive approach did not work. 

We can tokenize one article or all articles at once, it seems to work i.e. does not give an error.

In [50]:
tokenizer.pad_token = tokenizer.eos_token


In [31]:
tokenizedArticle = tokenizer(trainingArticles[1])

In [45]:
tokenizedArticlesAll = tokenizer(trainingArticles, padding=True, truncation=True, return_tensors="tf")

In [ ]:
batch = dict(tokenizer(trainingArticles, padding=True, truncation=True, return_tensors="tf"))

In [47]:
def tokenize_function(examples):
    return tokenizer(trainingArticles, padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

NameError: ignored

## Train the model

In [46]:
# train one batch to see if it works (it does not at the moment)
model.train_on_batch(tokenizedArticlesAll)

TypeError: ignored

In [51]:
model.train_on_batch(batch)

TypeError: ignored